In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression

In [2]:
### тест хи квадрат на нормальное, экспоненциальное, равномерное распределения
def my_chi2test(df1): 
  g = int(1 + np.floor(np.log2(len(df1))))
  h = (max(df1) - min(df1)) / g
  
  Z = [0] * (g + 1)
  for i in range(len(Z)):
    Z[i] = min(df1) + (h * i)
  data = [0] * g
  
  for i in range(len(df1)):
    for j in range(len(Z) - 1):
      if df1[i] >= Z[j] and df1[i] <= Z[j + 1]:
        data[j] += 1

  for i in range(len(data)):
    if data[i] == 0:
      data[i] = 1

  range_df = []
  mask = [round((Z[i] + Z[i + 1]) / 2, 2) for i in range(len(Z) - 1)]
  for i in range(len(data)):
    range_df += [mask[i]] * data[i]

  
  mn = 0
  chas = rel_freq(range_df)
  for i in range(len(chas)):
    if chas[i][0] != mask[i]:
      chas = (chas[0 : i] + [[mask[i], 1e-15]] + chas[i:])
      break

  mn = 0
  for i in range(len(chas)):
    mn += chas[i][0] * chas[i][1]

  vr = 0
  for i in range(len(chas)):
    vr += ((chas[i][0] - mn) ** 2) * chas[i][1]

  sq = np.sqrt(vr)
  print("N =", len(df1))
  print('mean =', round(mn, 3))
  print('sq = ', round(sq, 3))
  ###
  raspr = sorted([round(i, 3) for i in list(sq * np.random.randn(len(df1)) + mn)])
  
  n_chas = [0] * g
  for i in range(len(raspr)):
    for j in range(len(Z) - 1):
      if raspr[i] >= Z[j] and raspr[i] <= Z[j + 1]:
        n_chas[j] += 1
  
  test = chi2_contingency([data, n_chas])[0:2]
  print('NORMAL  chi: ', test[0], '  pvalue: ', test[1])
  ###
  uni = sorted([round(i, 3) for i in np.random.uniform(min(df1), max(df1), len(df1))])
  
  uni_chas = [0] * g
  for i in range(len(uni)):
    for j in range(len(Z) - 1):
      if uni[i] >= Z[j] and uni[i] <= Z[j + 1]:
        uni_chas[j] += 1

  
  uni_test = chi2_contingency([data, uni_chas])[0:2]
  print('UNIFORM  chi: ', uni_test[0], '  pvalue: ', uni_test[1])
  ###
  alfa = round(1/mn, 3)
  ex = sorted(np.random.exponential(np.abs(mn), len(df1)))

  ex_chas = [0] * g
  for i in range(len(ex)):
    for j in range(len(Z) - 1):
      if ex[i] >= Z[j] and ex[i] <= Z[j + 1]:
        ex_chas[j] += 1
  ex_test = chi2_contingency([data, ex_chas])[0:2]
  print('EXPONENTIAL  chi: ', ex_test[0], '  pvalue: ', ex_test[1])
  plt.hist(df1, bins = g)